In [9]:
import wandb
from wandb.keras import WandbCallback
wandb.init(project="CT-Unet-Train", entity="subinek")
#wandb.init(settings=wandb.Settings(start_method='thread'),project='CT-Unet-Training' )

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: subinek. Use `wandb login --relogin` to force relogin


In [1]:
import glob
import os

import tensorflow.keras
import numpy as np
import skimage
import SimpleITK as sitk

2022-11-05 13:30:03.405062: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-05 13:30:03.522825: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-05 13:30:04.005791: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-11-05 13:30:04.005863: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinf

In [2]:
#%% loading the libraries
from tensorflow.keras.layers import ConvLSTM2D, Bidirectional, BatchNormalization, Conv3D, Cropping3D, ZeroPadding3D, Activation, Input
from tensorflow.keras.layers import MaxPooling3D, UpSampling3D, Conv3DTranspose, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
import os
import sys
import numpy as np
import tensorflow as tf
from tensorflow import keras
from skimage import io
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
from utils import CTDatagenerator3D
from CTDatagenerator3D import DataGenerator
from LossMetrics import DiceLoss
from tensorflow.keras import regularizers

Num GPUs Available:  1


In [3]:
#train_gen.take(1)

In [4]:
        
train_params = {'dim': (256, 256, 32),
          'batch_size': 1,
          'im_path': './Data/Processed/Volumes/',
          'label_path': './Data/Processed/Labels/',
          'n_classes': 1,
          'shuffle': True,
          'augment': False}

train_gen = DataGenerator(**train_params)

In [5]:
val_params = {'dim': (256, 256, 32),
          'batch_size': 1,
          'im_path': './Data/Processed/Test/Volumes/',
          'label_path': './Data/Processed/Test/Labels/',
          'n_classes': 1,
          'shuffle': True,
          'augment': False}

val_gen = DataGenerator(**val_params)

In [6]:
import segmentation_models_3D as sm
sim_model = sm.Unet(
    'resnet18', 
    input_shape=(256, 256, 32, 1),
    encoder_weights=None
)
sim_model.summary()

Segmentation Models: using `tf.keras` framework.


2022-11-04 11:10:13.962840: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 data (InputLayer)              [(None, 256, 256, 3  0           []                               
                                2, 1)]                                                            
                                                                                                  
 bn_data (BatchNormalization)   (None, 256, 256, 32  3           ['data[0][0]']                   
                                , 1)                                                              
                                                                                                  
 zero_padding3d (ZeroPadding3D)  (None, 262, 262, 38  0          ['bn_data[0][0]']                
                                , 1)                                                        

                                                                                                  
 stage2_unit1_bn1 (BatchNormali  (None, 64, 64, 8, 6  256        ['add_1[0][0]']                  
 zation)                        4)                                                                
                                                                                                  
 stage2_unit1_relu1 (Activation  (None, 64, 64, 8, 6  0          ['stage2_unit1_bn1[0][0]']       
 )                              4)                                                                
                                                                                                  
 zero_padding3d_6 (ZeroPadding3  (None, 66, 66, 10,   0          ['stage2_unit1_relu1[0][0]']     
 D)                             64)                                                               
                                                                                                  
 stage2_un

                                56)                                                               
                                                                                                  
 add_4 (Add)                    (None, 16, 16, 2, 2  0           ['stage3_unit1_conv2[0][0]',     
                                56)                               'stage3_unit1_sc[0][0]']        
                                                                                                  
 stage3_unit2_bn1 (BatchNormali  (None, 16, 16, 2, 2  1024       ['add_4[0][0]']                  
 zation)                        56)                                                               
                                                                                                  
 stage3_unit2_relu1 (Activation  (None, 16, 16, 2, 2  0          ['stage3_unit2_bn1[0][0]']       
 )                              56)                                                               
          

 stage4_unit2_conv2 (Conv3D)    (None, 8, 8, 1, 512  7077888     ['zero_padding3d_17[0][0]']      
                                )                                                                 
                                                                                                  
 add_7 (Add)                    (None, 8, 8, 1, 512  0           ['stage4_unit2_conv2[0][0]',     
                                )                                 'add_6[0][0]']                  
                                                                                                  
 bn1 (BatchNormalization)       (None, 8, 8, 1, 512  2048        ['add_7[0][0]']                  
                                )                                                                 
                                                                                                  
 relu1 (Activation)             (None, 8, 8, 1, 512  0           ['bn1[0][0]']                    
          

                                                                                                  
 decoder_stage2b_relu (Activati  (None, 64, 64, 8, 6  0          ['decoder_stage2b_bn[0][0]']     
 on)                            4)                                                                
                                                                                                  
 decoder_stage3_upsampling (UpS  (None, 128, 128, 16  0          ['decoder_stage2b_relu[0][0]']   
 ampling3D)                     , 64)                                                             
                                                                                                  
 decoder_stage3_concat (Concate  (None, 128, 128, 16  0          ['decoder_stage3_upsampling[0][0]
 nate)                          , 128)                           ',                               
                                                                  'relu0[0][0]']                  
          

In [12]:
sim_model.compile(optimizer='adam', loss=DiceLoss, metrics = ['accuracy','mse'], run_eagerly=True)
weight_path="{}.hdf5".format('CT_100EP_Dice')

checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = False)

lr_ReduceLROnPlateau = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=10, min_lr=0.00001)

early_stop = EarlyStopping(monitor='loss', patience=50)

wandbCallback = WandbCallback(
    monitor="val_loss", verbose=0, mode="auto", save_weights_only=(False),
    log_weights=(False), log_gradients=(False), save_model=(False),
    training_data=None, validation_data=None, labels=[], predictions=36,
    generator=None, input_type=None, output_type=None, log_evaluation=(False),
    validation_steps=None, class_colors=None, log_batch_frequency=None,
    log_best_prefix="best_", save_graph=(True), validation_indexes=None,
    validation_row_processor=None, prediction_row_processor=None,
    infer_missing_processors=(True), log_evaluation_frequency=0,
    compute_flops=(False),
)

In [13]:
sim_model.fit(train_gen,
            epochs=100,
              batch_size = 1,
              validation_data = val_gen,
              callbacks=[checkpoint,lr_ReduceLROnPlateau, early_stop, wandbCallback ])

Epoch 1/100
38/38 [==============================] - ETA: 0s - loss: 0.6919 - accuracy: 0.8300 - mse: 0.1347 
Epoch 1: val_loss improved from inf to 0.99692, saving model to CT_100EP_Dice.hdf5
38/38 [==============================] - 758s 20s/step - loss: 0.6919 - accuracy: 0.8300 - mse: 0.1347 - val_loss: 0.9969 - val_accuracy: 0.8834 - val_mse: 0.1162 - lr: 0.0010
Epoch 2/100
38/38 [==============================] - ETA: 0s - loss: 0.7029 - accuracy: 0.8947 - mse: 0.0893 
Epoch 2: val_loss improved from 0.99692 to 0.98875, saving model to CT_100EP_Dice.hdf5
38/38 [==============================] - 744s 20s/step - loss: 0.7029 - accuracy: 0.8947 - mse: 0.0893 - val_loss: 0.9887 - val_accuracy: 0.8810 - val_mse: 0.1174 - lr: 0.0010
Epoch 3/100
38/38 [==============================] - ETA: 0s - loss: 0.5779 - accuracy: 0.9091 - mse: 0.0794 
Epoch 3: val_loss improved from 0.98875 to 0.98291, saving model to CT_100EP_Dice.hdf5
38/38 [==============================] - 754s 20s/step - loss

KeyboardInterrupt: 

### check the output

### predict output from the model

In [ ]:
val_params = {'dim': (256, 256, 32),
          'batch_size': 1,
          'im_path': './Data/Processed/Test/Volumes/',
          'label_path': './Data/Processed/Test/Labels/',
          'n_classes': 1,
          'shuffle': True,
          'augment': False}

val_gen = DataGenerator(**val_params)

In [4]:
output_model = tf.keras.models.load_model('./CT_750EP.hdf5', custom_objects={"DiceLoss":DiceLoss})
#output_model = sim_model
#output = output_model.predict(val_gen)
#output = np.squeeze(output)
#print(output.shape)

2022-11-05 13:30:10.964772: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-05 13:30:11.616670: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 908 MB memory:  -> device: 0, name: NVIDIA A30, pci bus id: 0000:c6:00.0, compute capability: 8.0


In [5]:
def generate_images(network_output):
    num_graphs = network_output.shape[2]
    for i in range(num_graphs):
        plt.clf()
        plt.title(i)
        plt.imshow(network_output[1,:,:,i],cmap='gray')
        plt.axis('off')
        plt.draw()
        plt.pause(0.01)
    
    
generate_images(output)

NameError: name 'output' is not defined

### Generate output for one sample

In [7]:
def generate_array_from_nrrd(path):
    img = sitk.ReadImage(path,sitk.sitkFloat32)
    img_arr = sitk.GetArrayFromImage(img)
    #img_arr = np.moveaxis(img_arr,[0],[2])
    img_arr = np.expand_dims(img_arr,  axis=-1)
    img_arr = np.expand_dims(img_arr, axis=0)
    print(img_arr.shape)
    return img_arr

In [8]:
volume = generate_array_from_nrrd('./Data/Processed/Test/Volumes/S11.nii.gz')
label = generate_array_from_nrrd('./Data/Processed/Test/Labels/S11.nii.gz')
network_output = output_model.predict(volume)

(1, 256, 256, 32, 1)
(1, 256, 256, 32, 1)


2022-11-05 13:30:32.257497: E tensorflow/stream_executor/cuda/cuda_dnn.cc:389] Could not create cudnn handle: CUDNN_STATUS_INTERNAL_ERROR
2022-11-05 13:30:32.257550: W ./tensorflow/stream_executor/stream.h:2104] attempting to perform DNN operation using StreamExecutor without DNN support


InternalError: Graph execution error:

Detected at node 'model_1/bn_data/FusedBatchNormV3' defined at (most recent call last):
    File "/home/erattakulangara/.conda/envs/unet3D_A30/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/erattakulangara/.conda/envs/unet3D_A30/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/erattakulangara/.conda/envs/unet3D_A30/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/erattakulangara/.conda/envs/unet3D_A30/lib/python3.9/site-packages/traitlets/config/application.py", line 976, in launch_instance
      app.start()
    File "/home/erattakulangara/.conda/envs/unet3D_A30/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/home/erattakulangara/.conda/envs/unet3D_A30/lib/python3.9/site-packages/tornado/platform/a

In [ ]:
slice_num = 15
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize = (20, 20))
ax1.imshow(np.squeeze(volume)[:,:,slice_num])
ax1.set_title('Input Image')
ax2.imshow(np.squeeze(network_output)[:,:,slice_num], cmap = 'gray')
ax2.set_title('Prediction')
ax3.imshow(np.squeeze(label)[:,:,slice_num], cmap = 'gray')
ax3.set_title('Actual Lung Volume')
fig.savefig('full_scan_prediction.png', dpi = 300)

In [ ]:
!pip list --format=freeze > unet3Drequirements.txt

In [ ]:
import os
print(os.getcwd())

In [ ]:
!pip install pyyaml

In [10]:
from scipy.spatial.distance import directed_hausdorff

In [11]:
directed_hausdorff(np.squeeze(volume), np.squeeze(label))[0]

ValueError: Buffer has wrong number of dimensions (expected 2, got 3)

Defaulting to user installation because normal site-packages is not writeable
  Using cached tensorflow_graphics-2021.12.3-py3-none-any.whl (4.3 MB)
  Using cached tensorflow_datasets-4.7.0-py3-none-any.whl (4.7 MB)
  Using cached tqdm-4.64.1-py2.py3-none-any.whl (78 kB)
  Using cached OpenEXR-1.3.9.tar.gz (13 kB)
  Preparing metadata (setup.py) ... done
  Using cached trimesh-3.16.0-py3-none-any.whl (660 kB)
  Using cached tensorflow_addons-0.18.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.1 MB)
  Using cached typeguard-2.13.3-py3-none-any.whl (17 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl (16 kB)
  Using cached etils-0.9.0-py3-none-any.whl (140 kB)
  Using cached tensorflow_metadata-1.10.0-py3-none-any.whl (50 kB)


  Using cached importlib_resources-5.10.0-py3-none-any.whl (34 kB)
  Using cached googleapis_common_protos-1.56.4-py2.py3-none-any.whl (211 kB)
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [16 lines of output]
      Looking for libOpenEXR...
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.linux-x86_64-3.9
      copying Imath.py -> build/lib.linux-x86_64-3.9
      running build_ext
      building 'OpenEXR' extension
      creating build/temp.linux-x86_64-3.9
      gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -fPIC -I/usr/include/OpenEXR -I/usr/local/include/OpenEXR -I/opt/local/include/OpenEXR -I/usr/include/Imath -I/usr/local/include/Imath -I/opt/local/include/Imath -I/usr/local/include/python3.9 -c OpenEXR.cpp -o build/temp.linux-x86_64-3.9/OpenEXR.o -g -DVERSION="1.3.9"
      OpenEXR.cpp:36:10: fatal error: Imf